<a href="https://colab.research.google.com/github/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/blob/main/Documentation/InteraktiveCamera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Grundlegende Idee**
Die Idee ist ein Zwei-Kamera-System zu einem Ein-Kamera-System umzugestalten. Dabei soll die erste Kamera auf einen Sprecher und die zweite Kamera auf ein Blatt Papier gerichtet sein. Sobald der Sprecher nun seinen Zeigefinger auf das Papier legt, wird dieses Kamerabild als Bild in Bild (pip) der ersten Kamerasicht angezeigt.


## Benötigte Hardware
Das System soll so einfach wie möglich sein und keine spezielle Hardware benötigen. Das heißt eine Handykamera, welche über Apps wie beispielsweise [Droid Cam](http://www.dev47apps.com/) auf Android einen Kamerafeed an den PC liefern sollen schon ausreichen. Dementsprechend soll es auch möglich sein dieses Handykamerabild innerhalb einer Bildschirmübertragung anzuzeigen, um so gezeigtes auf dem Bildschirm durch beispielsweise Handzeichnungen zu unterstützen.

## Funktionen
Wie bereits genannt wird der Nutzer zwei Bildinputs auswählen und festlegen müssen welche der Inputs das einzublendende Bild enthält. Dem Nutzer soll es möglich sein durch Zeigen mit dem Zeigefinger die Kamera zu Aktivieren und den Zeigefinger als Mittelpunkt des Bildes zu sehen. Durch eine "Raus-Zoom" Geste wie man sie von Smartphones kennt, wird ein digitaler Zoom ausführbar sein.

## Programmcode
Jeglicher Programmcode ist in dem folgende Repository zu finden:
[Medienverarbeitung Gruppe E - Interactive Camera](https://github.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system)

## Vorgehen im Projekt
Das Projekt wird als Projekttagebuch dokumentiert. Hier werden alle Ideen, Ansätze und Ergebnisse festgehalten. Zu Abschluss des Projekts wird das Gesamtergebnis noch einmal gesondert und aufbereitet präsentiert. 


# Projekttagebuch

## Erster Ansatz und Ergebnisse 
*11.11.20*

Zu erst kümmern wir uns um die Erkennung eines zeigenden Zeigefingers. Dafür trainieren wir ein Keras Modell mit Bildern, welche zeigende Zeigefinger (Bild 1), weiße (karrierte, linierte, blanko) Papiere (Bild 2) und andere Gesten wie flache Hände und Ähnliches (Bild 3) unterscheiden können soll.

![alt text](https://raw.githubusercontent.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/main/Documentation/Pictures/11.11.20/zu_unterscheidende_Objekte.png)

Das damit antrainierte Modell erkennt das gezeigte Bild und sortiert es der Rubrik ein, welche mit der höchsten Konfidenz übereinstimmt.

Die Verarbeitung des eingehenden Videostreams wird durch die Python-Bibliothek "[*OpenCV*](https://pypi.org/project/opencv-python/)" ermöglicht.

Durch diesen Ansatz konnten bisher erste Erfolge und enstehende Probleme verzeichnet werden

###Probleme
Es bisher Störfaktoren wie eine Computer-Maus (Bild 6) fälschlicherweise als ein Zeigen erkannt. Jedoch war es auch möglich in einer kontrollierten Umgebung einen Zeigefinger (Bild 4) von einem Blatt Papier (Bild 5) zu unterscheiden.
![alt text](https://raw.githubusercontent.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/main/Documentation/Pictures/11.11.20/Ergebnisse.png)



##Verwurf vorheriger Ergebnisse und entdeckung neuer Technologien
*18.11.2020*

Eine Objekterkennung durch ein neurales Netzwerk bedarf einer ausgesprochen Großen Datenmenge, welche kuriert wurden, um sie auf Relevante objekte zu reduzieren. Dies heißt in unserem Fall, dass wir einen für zwei Personen nicht erzeugbaren Datensatz an zeigenden Fingern und Hintergrundbildern benötigen würden, um akkurate Ergebnise zu erziehlen. Daher brauchen wir eine Alternative, welche mit einem realistischen Arbeitsaufwand zu bewältigen ist.

Nun stellt sich die Frage: Was macht einen zeigenden Finger aus? Oder noch simpler: Was macht eine Hand aus?

Wir als Menschen erkennen Objekte auf Distanzen primär über Form und Farbe. So entsteht die Idee, dass ein Computer ein Bild nach der Handfarbe eines Nutzers absuchen und diesen Bereich isolieren kann. Dies ist in "[*OpenCV*](https://pypi.org/project/opencv-python/)" über sogenannte Histogramme möglich. 
Die Grundlegende Idee ist die solche, dass anfangs die Hautfarbe des Nutzers durch eine simple Bildprobe ermittelt wird und so eine Hand (bzw. etwas Hautfarbenes) in einem Bild erkannt werden kann. Dabei besteht die Annahme, dass auch nur *eine* Hand im Bild zu sehen ist und keine weiteren mit der Hautfarbe übereinstimmenden Gegenstände im Bild sind.

![alt Text](https://raw.githubusercontent.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/main/Documentation/Pictures/18.11.20/Messung_Hautfarbe.png "Bild 7: Messpunkte auf Hand")

**Bild 7:** Messpunkte auf Hand




In Bild 7 ist eine solche Messung zu sehen. Die dabei markierten Rechtecke werden zu einem Bild zusammengefügt und aus diesem entstehenden Bild ein Histogram errechnet. Hier zu sehen ist, wie diese Regionen markiert und für die Messung genutzt werden:


```python
def drawMeasuringRectangles(frame):
    """Draws 'amountOfMeasuringRectangles' Rectangles on the given frame and returns the modified image"""
    rows, cols, dontCare = frame.shape
    global amountOfMeasuringRectangles, xCoordinatesOfMeasuringRectangles_topLeft, yCoordinatesOfMeasuringRectangles_topLeft, xCoordinatesOfMeasuringRectangles_bottomRight, yCoordinatesOfMeasuringRectangles_bottomRight

    # position messure points of hand histogram
    xCoordinatesOfMeasuringRectangles_topLeft = np.array(
        [6 * rows / 20, 6 * rows / 20, 6 * rows / 20, 9 * rows / 20, 9 * rows / 20, 9 * rows / 20, 12 * rows / 20,
         12 * rows / 20, 12 * rows / 20], dtype=np.uint32)

    yCoordinatesOfMeasuringRectangles_topLeft = np.array(
        [9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20,
         10 * cols / 20, 11 * cols / 20], dtype=np.uint32)

    # define shape of drawn small rectangles | here 10x10
    xCoordinatesOfMeasuringRectangles_bottomRight = xCoordinatesOfMeasuringRectangles_topLeft + 10
    yCoordinatesOfMeasuringRectangles_bottomRight = yCoordinatesOfMeasuringRectangles_topLeft + 10

    # draw calculated rectangles
    for i in range(amountOfMeasuringRectangles):
        cv2.rectangle(frame,
                      (yCoordinatesOfMeasuringRectangles_topLeft[i], xCoordinatesOfMeasuringRectangles_topLeft[i]),
                      (yCoordinatesOfMeasuringRectangles_bottomRight[i],
                       xCoordinatesOfMeasuringRectangles_bottomRight[i]),
                      (0, 255, 0), 1)

    return frame

  def createHandHistogram(frame):
    global xCoordinatesOfMeasuringRectangles_topLeft, yCoordinatesOfMeasuringRectangles_topLeft

    # convert cv2 bgr colorspace to hsv colorspace for easier handling
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # create new blank Region Of Interest matrix/image
    roi = np.zeros([90, 10, 3], dtype=hsv_frame.dtype)

    # fill ROI with the sample rectangles
    for i in range(amountOfMeasuringRectangles):
        roi[i * 10: i * 10 + 10, 0: 10] = hsv_frame[xCoordinatesOfMeasuringRectangles_topLeft[i]:
                                                    xCoordinatesOfMeasuringRectangles_topLeft[i] + 10,
                                          yCoordinatesOfMeasuringRectangles_topLeft[i]:
                                          yCoordinatesOfMeasuringRectangles_topLeft[i] + 10]

    # create a Hand histogram and normalize it
    hand_hist = cv2.calcHist([roi], [0, 1], None, [180, 256], [0, 180, 0, 256])

    # remove noise and retun
    return cv2.normalize(hand_hist, hand_hist, 0, 255, cv2.NORM_MINMAX)
```
Das daraus entstehende Histogram wird verwendet, um auf das gegebene Bild bzw. den Videoframe einen Graustufenfilter anzuwenden. Dabei stimmen hellere (weißere) Pixel mehr mit dem errechneten Histogram überein, während dunklere (schwarze) Pixel nicht über einstimmen und so eine andere Farbe vorweisen.